# (Developers Code) - Finger Detection and Counting using OpenCV

In [1]:
# importing libraries
import cv2
import numpy as np
from sklearn.metrics import pairwise

## Global Variables

We will use these as we go along.

In [2]:
background = None

# Start with a halfway point between 0 and 1 of accumulated weight
accumulated_weight = 0.5

# Coordinates of rectangle of ROI for grabbing the hand 
roi_top = 20
roi_bottom = 300
roi_right = 30
roi_left = 300

## Average Background Value of ROI

The function calculates the weighted sum of the input image src and the accumulator dst so that dst becomes a running average of a frame sequence:

In [3]:
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    # for the first iteration, create the background from a copy of the frame
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    # compute weighted average, accumulate it, and take its average
    cv2.accumulateWeighted(frame,background,accumulated_weight)

## Segmenting the Hand Region in Frame

In [4]:
def segment(frame,threshold_min=10):
    
    # Calculates the absolute difference between the averaged background obtained and the input frame passed
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    # Applying a threshold to the image so as to extract out the hand from foreground
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    # Grabing the external contours from the image
    contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # if the length of contours list = 0, then we didn't find any contours
    if len(contours) == 0:
        return None
    
    else:
        # ASSUMING THE LARGEST EXTERNAL CONTOUR IN ROI, IS THE HAND
        # this will be our segment
        hand_segment = max(contours,key=cv2.contourArea)


        # Return both the hand segment and the thresholded hand image
        return (diff, thresholded,hand_segment)

## Counting Fingers with a Convex Hull

We have calculated the external contour of the hand. Now using that segmented hand, we will make a convex hull around it to find how many fingers are up 



In [5]:
def count_fingers(thresholded,hand_segment, percent, thick):
    
    # Calculated the convex hull of the hand segment
    conv_hull =cv2.convexHull(hand_segment)
    
    
    # Now the convex hull will have at least 4 most outward points, on the top, bottom, left, and right.
    # Grabbing those points using argmin and argmax

    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    # Transforming rotated rectangle into rectangle normal to axis
    tlx, trx, brx, blx = top[0], right[0], bottom[0], left[0]
    tly, tryy, bry, bly = top[1], right[1], bottom[1], left[1]
    
    
    x1=min(tlx,trx,brx,blx)    # top-left pt. is the uppermost of the 4 points
    x2=max(tlx,trx,brx,blx)    # bottom-right pt. is the lowermost of the 4 points
    y1=min(tly,tryy,bry,bly)   # top-right pt. is the rightmost of the 4 points
    y2=max(tly,tryy,bry,bly)   # bottom-left pt. is the leftmost of the 4 points
    
    # creating a rectangle of convex hull
    hull_rect  = cv2.rectangle(thresholded.copy(), (x1, y1), (x2, y2), (255), 3)
    
    hull_rect = cv2.merge((hull_rect, hull_rect, hull_rect))
    
    # Drawing circles at extreme points
    for corner in [top, left, bottom, right]: 
        hull_rect = cv2.circle(hull_rect, corner, 8, (0, 0, 255), 4)


    
    # Assuming the center of hand to be at the mid-point of (top and bottom) and (left and right)
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    # Calculate the Euclidean Distance between the center of the hand and the left, right, top, and bottom.    
    distance = pairwise.euclidean_distances([(cX,cY)],Y=[left,right,top,bottom])[0]
    
    # Drawing lines from center of hand to the extreme points 
    lines = hull_rect.copy()
    for line in [top, bottom, left, right]:
        lines = cv2.line(lines, (cX, cY), line, (255,0,0), 2)
    
        
    # Grab the largest distance
    max_distance = distance.max()
    cv2.putText(lines, f'{max_distance}', (5,30), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255, 255), 1)

    # Creating a circle with 90% radius if the max euclidean distance
    radius = int(percent/100*max_distance)
    circumfrence = (2*np.pi*radius)

    # Not grab an ROI of only that circle
    circular_roi = np.zeros(thresholded.shape[:2],dtype='uint8')
    
    # draw the circular ROI    
    cv2.circle(circular_roi,(cX,cY),radius,255,thick)
    
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    # Grab contours in circle ROI    
    contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0

    # loop through the contours to to count fingers.    
    for cnt in contours:
        
        # Bounding box of countour
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # Increment count of fingers based on two conditions:
            # we are excluding the bottom area of hand to be counter as finger
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
            # Number of points along the countous does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points and max_distance > 92:
            count += 1
            
    return count, hull_rect, lines, circular_roi

## Stacking preview images 

This function stacks together individual images and returns as one big image<br>
Scaling factor and Array of images are passed as inputs

In [6]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver


## Main


In [7]:
def empty(a):
    pass

cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 640, 240)
cv2.createTrackbar("percent", "TrackBars", 90, 100, empty)
cv2.createTrackbar("thickness", "TrackBars", 10, 30, empty)



cam = cv2.VideoCapture(0)

num_frames = 0
hand = None
while True:
    p = cv2.getTrackbarPos("percent", "TrackBars")
    t = cv2.getTrackbarPos("thickness", "TrackBars")

    ret, frame = cam.read()
    frame_copy = frame.copy()

    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT... GETTING BACKGROUND',(5,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1)
#            cv2.imshow('Finger Count',frame_copy)
    else:
        
        hand = segment(gray)
       
        if hand is not None:
#            cv2.imshow('hand',hand[0])
            diff, thresholded , hand_segment = hand
#            cv2.imshow('diff',diff)
            
            # DRAWS CONTOURS AROUND REAL HAND IN LIVE STREAM
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            fingers, hull_rect, lines, circular_roi = count_fingers(thresholded,hand_segment, p, t)
            
#             cv2.imshow("hull_rect", hull_rect)
#             cv2.imshow("lines", lines)
#             cv2.imshow("circular_roi", circular_roi)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
#            cv2.imshow('Thresholded',thresholded)
            
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    num_frames += 1
    
    if hand is not None:
        imgStack = stackImages(1.2, ([diff, thresholded, hull_rect], [lines, circular_roi, frame_copy]))
        cv2.imshow('Stack', imgStack)

    cv2.imshow('Finger Count',frame_copy)
    
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

